# **Instructions**

This code has two purposes:  
1) Import an existing CSV file from bibliometrix, drop duplicates by the DOI identifier (and not loose nan values), and then clean the following columns:  
- Author keywords: All existing keywords are exported to a "csv" to cluster different spellings of each important term. This then feeds back to the database through a separate excel file that must be compile by the user "Checklist_DE.xlsx" following the stablished format (old value - new value).
- Sources: All existing sources are exported to a "csv" to cluster different spellings of each important term. This then feeds back to the database through a separate excel file that must be compile by the user "CheckList_SO.xlsx" following the stablished format (old value - new value).
- First author country:

## **I - Import**

### **I.I - New Database from Bibliometrix**
We import the Dataframe (df) compiled through Bibliometrix.

In [1]:
api = input("Insert API key from Plotly: ")
user = input("Insert user name from Plotly: ")
# 2NexotbGiaTPZ5x7CNWa; mlcanales
%config InlineBackend.figure_format ='retina'
import pandas as pd
import os
import plotly
import chart_studio
import plotly.io as pio
import chart_studio.plotly as py 
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import matplotlib.cm
from plotly.offline import plot
from glob import glob
import shutil
chart_studio.tools.set_credentials_file(username=user, api_key=api)


# To get all results printed: (or "last_expr" to only last expression from cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Insert API key from Plotly:  2NexotbGiaTPZ5x7CNWa
Insert user name from Plotly:  mlcanales


In [2]:
# We start from the Jupyter notebooks folder
os.getcwd()
# Move back to main path: Bibliometric analysis
%cd ..
main_path = os.getcwd() + "/"

'/Users/macadmin/Google Drive/Shared Folders/GW-ABM — Review/Bibliometric analysis/Jupyter notebooks'

/Users/macadmin/Google Drive/Shared Folders/GW-ABM — Review/Bibliometric analysis


In [ ]:
df = pd.read_csv('Merged_dataframe_2.csv')
df.shape
print(df.isna().sum())

In [3]:
df.shape

(1055, 37)

### **I.II - Updated Database from Bilbiometrix**  
Here we import the updated database through bibliometrix

In [ ]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df2 = pd.read_csv("Updated_Database.csv")
#print(df2.isna().sum())
#df2
df2 = df2.drop('Unnamed: 0', 1)
#df2
df = df2.copy()
df.shape

## **II - Oficial Cleanup**

We drop duplicates by column "DI" from the .csv (due to nan values, this step must be done in python)

In [5]:
print("Database dimensions are: ", df.shape)

# Series of "DI" data from dataframe
s = df.loc[:,['DI']]

# Dropping NaN values (so they don't get dropped in duplicated detection)
s = s.dropna()

# Leaving only duplicated values
s = s.duplicated()
s = s[s == True] 

# Geting list of index values to drop in original dataframe
s_index = s.index.tolist()
print("Indexes removed from the dataframe are: ", s_index)
list2 = []
for element in s_index:
    list2.append(df.loc[element,['TI']])
serie = pd.Series(list2)
serie

# Editing original dataframe and reseting index (drop=true avoids adding the old index as column)
df = df.drop(s_index)
df = df.reset_index(drop=True)

print("Database dimensions are: ", df.shape)

Database dimensions are:  (1055, 37)
Indexes removed from the dataframe are:  [545, 562, 568, 595, 596, 620, 634, 669, 670, 681, 715, 741, 820, 844, 894, 1038, 1047, 1050]


0     TI    INVESTIGATING THE IMPACT OF RIVER FLOODS...
1     TI    ASSESSING VULNERABILITY TO FLOODING USIN...
2     TI    MONITORING OF FLOOD EMBANKMENTS WITH THE...
3     TI    HYDROSEDIMENTARY CONNECTIVITY IN A SMALL...
4     TI    REASONING ABOUT RIVER BASINS WAWO REVISI...
5     TI    AGENT BASED MODELLING FORWATER RESOURCE ...
6     TI    MEASURING THE IMPACT OF LINEAR LANDSCAPE...
7     TI    SIMULATING STAKEHOLDER BEHAVIOR IN A MAR...
8     TI    AGENTBASED SIMULATION APPROACH FOR MANAG...
9     TI    ASSESSING GROUNDWATER POLICY WITH COUPLE...
10    TI    COMPLEMENTARY METHODS TO PLAN PEDESTRIAN...
11    TI    UBIQUITOUS MULTIAGENT ENVIRONMENTAL HAZA...
12    TI    MODELLING WITH STAKEHOLDERS TO INTEGRATE...
13    TI    LAND MARKET INTERACTIONS BETWEEN HETEROG...
14    TI    THE USE OF AGENT BASED MODELLING TECHIQU...
15    TI    APPLICATION OF AN EULERIANLAGRANGLANAGEN...
16    TI    MODELLING OF THE CONSTRUCTION OF THE RHO...
17    TI    A NEW APPROACH FOR MODELLING SEDIMEN

Database dimensions are:  (1037, 37)


## **Database Path selection (GWABM - SH - TC - PM - EM - EM2)**

In [9]:
# Here we create the variable db_path used to direct the analysis to each specific component
db = input("Select from GWABM - SH - TC - PM - EM - EM2 - Floods: ")
db_path = "Cleaning Files " + db + "/"                   
print("Writing cleaning files to: ", db_path)
print("Comparison Files are used on each subsection")

Select from GWABM - SH - TC - PM - EM - EM2 - Floods:  Floods


Writing cleaning files to:  Cleaning Files Floods/
Comparison Files are used on each subsection


## **Author Keywords (DE) &#9745;**
**DE column extracted from dataframe to single csv file with non-duplicated keywords. CheckList generated by hand from this in excel. Dataframe is modified directly reading the CheckList**  
The main idea is to detect different spellings of some keywords, which are stated in the CheckList file by reviewing the "csv" file generated first here. This CheckList works as a guide to re-edit the dataframe in-situ.

In [10]:
# First we clean the Sources column from "/&" and "&"" characters
df['DE'] = df['DE'].str.replace('\&', 'AND', regex=False)
df['DE'] = df['DE'].str.replace('&', 'AND', regex=False)

# We extract a list with keywords from the dataframe
df_list = df['DE'].tolist()
# NaN values are erased
df_list_cleaned = [str(x) for x in df_list if str(x) != 'nan']
# The list is splitted since there are several keywords in each row of the dataframe
full_list = []
for element in df_list_cleaned:
    full_list.extend(element.split(';'))

# We save the keywords to series (s1) and remove spacings in words
s1 = pd.Series(full_list)
print("Ammount of Author Keywords detected are: ", s1.shape)
s1 = s1.str.strip()

# We generate a dataframe from this column and then add a new column with co-occurrences counting
dataframe = pd.DataFrame(columns = ['DE'])
dataframe['DE'] = s1
print("Dataframe generated from author keywords with dimensions: ", dataframe.shape)
dataframe = dataframe.sort_values(by=['DE'])
dataframe = dataframe.reset_index(drop=True)
dataframe_test = pd.DataFrame({'Count' : dataframe.groupby( ["DE"] ).size()}).reset_index()
print("Dataframe with co-occurrences now has the following dimensions: ", dataframe_test.shape)
#dataframe_test

# The final author keywords are exported to ".csv" to detect and cluster similar ones and compile a CheckList by hand
e_path = db_path + 'Spelling_Check_DE.xlsx'
dataframe_test.to_excel(e_path, header=True, index = False)

Ammount of Author Keywords detected are:  (3879,)
Dataframe generated from author keywords with dimensions:  (3879, 1)
Dataframe with co-occurrences now has the following dimensions:  (2529, 2)


> Check "Spelling_Check_DE.xlsx" file and write over "CheckList_DE.xlsx" the new keywords 

In [29]:
# After the "CheckList_DE.xlsx" has been created (with the 2 columns format: old value - new value), it is used to edit the dataframe
e_path = "Comparison Files/CheckList_DE.xlsx"
check_list = pd.read_excel(e_path)
# First we get rid of NaN values replacing them by an empty string ""
df['DE'] = df['DE'].fillna("")
# Each row of the selected column ("DE") is compiled into list1, cleaned in list2, and repaste to dataframe
list1 = []
list2 = []
for index, row in df.iterrows():
    list1.extend(row['DE'].split(';'))
    for element in list1:
        list2.append(element.strip())
    # Here list2 is a list with the first row of the dataframe cleaned from spaces and ready for comparison:
    i = 0
    for element in list2:
        for index, column in check_list.iterrows():
            if element == column[0]:
                list2[i] = column[1]
                break
        i = i + 1
    #Here list2 has been transformed with specific "terms" listed in check_list
    #Now we must add it back to original dataframe
    df.replace(row['DE'], "; ".join(list2), inplace = True)
    #And finally clean all values for next row in dataframe
    list1 = []
    list2 = []

In [ ]:
# To check we print the Author Keywords column and visually check it has been edited properly according to the Excel file designed and imported in previous cell
print(df.isna().sum())
df['DE']
# To check a particular cell value:
#df.loc[[45], ['DE']]


## **Sources (SO) &#9745;**
**SO column extracted from dataframe to single csv file with non-duplicated keywords. CheckList generated by hand from this in excel. Dataframe is modified directly reading the CheckList**  
The main idea is to extract the Sources stored in the dataframe into a "csv" which does not stores "nan" or duplicates. Then, manually similar spellings of the same source can be detected and pasted in the CheckList file which acts as a guide for the dataframe edition.

In [ ]:
# First we clean the Sources column from "/&" and "&"" characters
df['SO'] = df['SO'].str.replace('\&', 'AND', regex=False)
df['SO'] = df['SO'].str.replace('&', 'AND', regex=False)

# We extract a list with sources from the dataframe
df_list = df['SO'].tolist()
# NaN values are erased
df_list_cleaned = [str(x) for x in df_list if str(x) != 'nan']

# We save the sources to series (s1) and remove spacings in words
s1 = pd.Series(df_list_cleaned)
print("Ammount of sources detected are: ", s1.shape)
s1 = s1.str.strip()

# We generate a dataframe from this column and then add a new column with co-occurrences counting
dataframe = pd.DataFrame(columns = ['SO'])
dataframe['SO'] = s1

print("Dataframe generated from Sources with dimensions: ", dataframe.shape)
dataframe = dataframe.sort_values(by=['SO'])
dataframe = dataframe.reset_index(drop=True)
dataframe_test = pd.DataFrame({'Count' : dataframe.groupby( ["SO"] ).size()}).reset_index()
print("Dataframe with co-occurrences of sources now has the following dimensions: ", dataframe_test.shape)
#dataframe_test

# The final sources are exported to ".csv" to detect and cluster similar ones and compile a CheckList by hand
e_path = db_path + 'Spelling_Check_SO.xlsx'
dataframe_test.to_excel(e_path, header=True, index = False)

> Check the "Spelling_Check_SO.xlsx" file for sources to re-write (mispelled or different spellings) and write "CheckList_SO.xlsx" in the format explicited.

In [ ]:
e_path = 'Comparison Files/CheckList_SO.xlsx'
check_list = pd.read_excel(e_path) #Follow same 2 column format: old value - new value

# First we get rid of NaN values replacing them by an empty string ""
df['SO'] = df['SO'].fillna("")
# Each row of the selected column ("SO") is compiled into list1, cleaned in list2, and repaste to dataframe
list1 = []
list2 = []
for index, row in df.iterrows():
    list1.extend(row['SO'].split(';'))
    for element in list1:
        list2.append(element.strip())
    # Here list2 is a list with the first row of the dataframe cleaned from spaces and ready for comparison:
    i = 0
    for element in list2:
        for index, column in check_list.iterrows():
            if element == column[0]:
                list2[i] = column[1]
                break
        i = i + 1
    #Here list2 has been transformed with specific "terms" listed in check_list
    #Now we must add it back to original dataframe
    df.replace(row['SO'], ", ".join(list2), inplace = True)
    #And finally clean all values for next row in dataframe
    list1 = []
    list2 = []

In [ ]:
# To check we print the Author Keywords column and visually check it has been edited properly according to the Excel file designed and imported in previous cell
df['SO']
# To check a particular cell value:
#df.loc[[45], ['SO']]

## **Author Countries (AU1_CO) &#9745;**
**AU1_CO column extracted from dataframe to single csv file with non-duplicated keywords. CheckList generated by hand from this in excel. Dataframe is modified directly reading the CheckList**  
The main idea is to extract the Country of Affiliation of each first author stored in the dataframe, and detect all "nan" values, so that they can then be updated into the dataframe. Also, it is important to check whether there is different spellings of the same country. The Co-authors country of affiliation is not checked here.

In [ ]:
# Getting a list of Country of Affiliation of each first author (AU1_CO) and co-author
# Here, the idea is to check if all countries under AU1_CO are written correctily (i.e. USA vs United States) to group them correctly.
df_sliced = df[['DI', 'TI', 'AU', 'AU1_CO', 'AU_CO']]

# First for spelling issues we check the different countries listed in column AU1_CO
df_not_null = df_sliced.dropna(subset = ['AU1_CO'])
df_not_null = df_not_null.sort_values(by =['AU1_CO'])
df_not_null = df_not_null.drop_duplicates(subset = "AU1_CO")

e_path = db_path + 'Spelling_Check_AU1_CO.xlsx'
df_not_null.to_excel(e_path)

In [ ]:
# Now, to add the missing countries, we will have to detect them, saved them to excel and add them manually by looking at WOS or Scopus
df_null_1 = df_sliced[df["AU1_CO"].isnull()]
df_null_1 = df_null_1.reindex(columns=['DI', 'TI', 'AU', 'AU1_CO', 'AU_CO', 'AU1_CO_NEW'], fill_value = "")

e_path = db_path + 'NaN_Check_AU1_CO.xlsx'
df_null_1.to_excel(e_path)

> Check the "Spelling_Check_AU1_CO.xlsx" file for mispelled elements and the "NaN_Check_AU1_CO.xlsx" for NaN values. Replaces for both are added to "CheckList_AU1_CO.xlsx" in the "AU1_CO_NEW" column.

In [ ]:
# After manually adding in the column "AU1_CO_NEW" the respective countries needed, we save the excel as "NaN_Check_AU1_CO_IN.xlsx" to finally edit "df"
e_path = db_path + 'CheckList_AU1_CO.xlsx'
check_list = pd.read_excel(e_path) #Follow same format: old value - new value

# First we get rid of NaN values replacing them by an empty string "" in df
df['AU1_CO'] = df['AU1_CO'].fillna("")
# For each row of the selected column ("AU1_CO") of df, we compare the NaN check excel through the DI column and edit its values.
for index1, row in df.iterrows():
    for index2, column in check_list.iterrows():
        if column[0] == index1:
            df.loc[index1,'AU1_CO'] = column[6]
            break


In [ ]:
        """if column[1] == row['DI']:
            df.loc[index1,'AU1_CO'] = column[6]
            #df.replace(row[34], column[6], inplace = True)
            break"""

In [ ]:
df['AU1_CO']

## **III.I - GW_ABM Database Specific Filtering &#9745;**
**Here we manually erase those documents not needed for our analysis**  

In [ ]:
# We first identify by visualizing the dataframe, those papers to erase and add them to df_eliminated by filtering thorugh the DI column
df_eliminated = df.loc[df['DI'] == "10.1016/S1251-8050(99)80113-X"]
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.1046/J.1365-3091.2001.00419.X"])
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.1016/J.GEOMORPH.2015.12.018"])
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.1016/J.SNB.2017.04.026"])
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.1016/J.WSE.2018.07.006"])
df_eliminated = df_eliminated.append(df.iloc[df[df.TI == 'REGIONALSUBREGIONAL 2D3D APPROACH TO MODEL GROUNDWATER FLOW AT SANDIA NATIONAL LABORATORIES ALBUQUERQUE NEW MEXICO USA'].index])
df_eliminated = df_eliminated.append(df.iloc[df[df.TI == 'EVOLUTIONARY MODEL OF COAL MINE WATER HAZARDS BASED ON MULTIAGENT SIMULATION'].index])
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.1180/CLAYMIN.2013.048.2.05"])
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.1007/S10653-013-9531-1"])
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.1371/JOURNAL.PONE.0150626"])
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.1016/J.APGEOCHEM.2016.07.005"])
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.12011/1000-6788(2017)12-3289-08"])
df_eliminated = df_eliminated.append(df.loc[df['DI'] == "10.11975/J.ISSN.1002-6819.2019.11.023"])
df_eliminated = df_eliminated.append(df.iloc[df[df.TI == 'PURPOSE PROCESSES PARTNERSHIPS AND PRODUCTS FOUR PS TO ADVANCE PARTICIPATORY SOCIOENVIRONMENTAL MODELING'].index])
list_to_erase = df_eliminated.index.tolist()
df.shape
df = df.drop(list_to_erase)
df = df.reset_index(drop=True)
df.shape

In [ ]:
# Now, after assessing through zotero, we eliminate all documents with no groundwater modelled or human behaviour
# Always look for the DI or the Title in the dataframe, since there is where python will look for
# df[['TI', 'DI']]
df_eliminated2 = df.loc[df['DI'] == "10.1080/1747423X.2018.1499828"]
df_eliminated2 = df_eliminated2.append(df.loc[df['DI'] == "10.1016/J.JHYDROL.2012.02.032"])
df_eliminated2 = df_eliminated2.append(df.loc[df['DI'] == "10.1016/J.JCLEPRO.2018.12.065"])
df_eliminated2 = df_eliminated2.append(df.loc[df['DI'] == "10.1016/J.AGEE.2004.08.007"])
df_eliminated2 = df_eliminated2.append(df.loc[df['DI'] == "10.1002/2014WR016825"])
df_eliminated2 = df_eliminated2.append(df.loc[df['DI'] == "10.5751/ES-08642-210323"])
df_eliminated2 = df_eliminated2.append(df.iloc[df[df.TI == 'MODELING LEARNING AND PLANNING TOGETHER AN APPLICATION OF PARTICIPATORY AGENTBASED MODELING TO ENVIRONMENTAL PLANNING'].index])
df_eliminated2 = df_eliminated2.append(df.loc[df['DI'] == "10.1002/EAP.1627"])
list_to_erase = df_eliminated2.index.tolist()
df.shape
df = df.drop(list_to_erase)
df = df.reset_index(drop=True)
df.shape

## **III.II - SH Database Specific Filtering**
**Here we manually assign Publication Years missing in the database**  

In [ ]:
# First we detect NaN values on PY
PY_nulls_list = SH.loc[SH['PY'].isnull()].index
PY_nulls_list
for i in PY_nulls_list:
    print(SH.loc[i, ['PY', 'TI']])

# Changes
PY_new = [2019, 2019, 2019]
j = 0
for i in PY_nulls_list:
    SH.loc[i, 'PY'] = PY_new[j]
    j+=1

# Revision
for i in PY_nulls_list:
    print(SH.loc[i, ['PY', 'TI']])

In [ ]:
df.shape
df.loc[df['SO']=="AMBIO"]
df.loc[256, ['TI']]

## **III.III - TC Database Specific Filtering**
**Here we manually assign Publication Years missing in the database**  

In [ ]:
# First we detect NaN values on PY
PY_nulls_list = TC.loc[TC['PY'].isnull()].index
PY_nulls_list
for i in PY_nulls_list:
    print(TC.loc[i, ['PY', 'TI']])

# Changes
PY_new = [2019]
j = 0
for i in PY_nulls_list:
    TC.loc[i, 'PY'] = PY_new[j]
    j+=1

# Revision
for i in PY_nulls_list:
    print(TC.loc[i, ['PY', 'TI']])

## **III.IV - PM Database Specific Filtering**
**Here we manually assign Publication Years missing in the database**  

In [ ]:
# First we detect NaN values on PY
PY_nulls_list = PM.loc[PM['PY'].isnull()].index
PY_nulls_list
for i in PY_nulls_list:
    print(PM.loc[i, ['PY', 'TI']])

# Changes
PY_new = [2019, 2019, 2019]
j = 0
for i in PY_nulls_list:
    PM.loc[i, 'PY'] = PY_new[j]
    j+=1

# Revision
for i in PY_nulls_list:
    print(PM.loc[i, ['PY', 'TI']])

## **III.V - EM Database Specific Filtering**
**Here we manually erase those documents not needed for our analysis**  

In [ ]:
# First we detect NaN values
PY_nulls_list = EM.loc[EM['PY'].isnull()].index
PY_nulls_list
for i in PY_nulls_list:
    print(EM.loc[i, ['PY', 'TI']])

# Changes:
PY_new = [2019]
j = 0
for i in PY_nulls_list:
    EM.loc[i, 'PY'] = PY_new[j]
    j+=1
# Revision
for i in PY_nulls_list:
    print(EM.loc[i, ['PY', 'TI']])    

## **III.VI - EM_2 Database Specific Filtering**
**Here we manually erase those documents not needed for our analysis**  

In [ ]:
# First we detect NaN values
PY_nulls_list = EM_2.loc[EM_2['PY'].isnull()].index
PY_nulls_list
for i in PY_nulls_list:
    print(EM_2.loc[i, ['PY', 'TI']])

# Changes:
PY_new = [2019, 2019, 2019, 2019, 2019, 2019]
j = 0
for i in PY_nulls_list:
    EM_2.loc[i, 'PY'] = PY_new[j]
    j+=1
# Revision
for i in PY_nulls_list:
    print(EM_2.loc[i, ['PY', 'TI']])    

## **Databases**  
**Each database iteration is saved over these temporary variables as dataframes and csv files, ready for using in bibliometrix**

### **GW_ABM DATABASE &#9745;**

In [ ]:
#GW_ABM = df.copy()
print('GWABM Database dimensions are: ', GW_ABM.shape)
e_path = "Cleaning Files GWABM/GWABM_DB.csv"
GW_ABM.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

### **SOCIOHYDROLOGY DATABASE &#9745;**

In [ ]:
#SH = df.copy()
print('SH Database dimensions are: ', SH.shape)
e_path = "Cleaning Files SH/SH_DB.csv"
SH.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

### **TELECOUPLING DATABASE &#9745;**

In [ ]:
#TC = df.copy()
print('TC Database dimensions are: ', TC.shape)
e_path = "Cleaning Files TC/TC_DB.csv"
TC.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

### **PARTICIPATORY MODELLING DATABASE &#9745;**

In [ ]:
#PM = df.copy()
print('PM Database dimensions are: ', PM.shape)
e_path = "Cleaning Files PM/PM_DB.csv"
PM.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

### **EXPLORATORY MODELLING DATABASE &#9745;**

In [ ]:
#EM = df.copy()
print('EM Database dimensions are: ', EM.shape)
e_path = "Cleaning Files EM/EM_DB.csv"
EM.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

### **EXPLORATORY MODELLING WITH DEEP UNCERTAINTY AND ROBUST DECISION MAKING &#9745;**

In [ ]:
#EM_2 = df.copy()
print('EM_2 (with "robust" and "deep uncertainty") Database dimensions are: ', EM_2.shape)
e_path = "Cleaning Files EM2/EM2_DB.csv"
EM_2.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

### **Other Databases**

#### **Sociohydrology and Groundwater &#9745;**

In [ ]:
SH_GW = df.copy()
print('SH_GW Database dimensions are: ', SH_GW.shape)
e_path = "GW Related Frameworks/SH_GW_DB.csv"
SH_GW.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

In [ ]:
# Specific filtering: Here we erase documents before 2012
SH_GW["PY"].sort_values()
#SH_GW.drop([46,52,45,44], inplace=True)
SH_GW.reset_index(drop=True)
SH_GW["PY"].sort_values()

#### **Telecoupling and Groundwater &#9745;**

In [ ]:
TC_GW = df.copy()
print('TC_GW Database dimensions are: ', TC_GW.shape)
e_path = "GW Related Frameworks/TC_GW_DB.csv"
TC_GW.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

#### **Participatory Modelling and Groundwater &#9745;**

In [ ]:
PM_GW = df.copy()
print('PM_GW Database dimensions are: ', PM_GW.shape)
e_path = "GW Related Frameworks/PM_GW_DB.csv"
PM_GW.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

#### **Exploratory Modelling and Groundwater &#9745;**

In [ ]:
EM_GW = df.copy()
print('EM_GW (with "robust" and "deep uncertainty") Database dimensions are: ', EM_GW.shape)
e_path = "GW Related Frameworks/EM_GW_DB.csv"
EM_GW.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

#### **Flooding and ABM &#9745;**

In [30]:
FL_ABM = df.copy()
print('FL_ABM Database dimensions are: ', FL_ABM.shape)
e_path = "Cleaning Files Floods/FL_ABM_DB.csv"
FL_ABM.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

FL_ABM Database dimensions are:  (1037, 37)


## **Databases Combination**  
**Cleaned Databases can be combined here before a final export to bibliometrix**

In [3]:
GW_ABM = pd.read_csv("Keyword Searches/Main DB (ABM & GW)/Database/Main DB (ABM & GW).csv")
SH = pd.read_csv("Keyword Searches/SH/Database/SH.csv")
TC = pd.read_csv("Keyword Searches/TC/Database/TC.csv")
PM = pd.read_csv("Keyword Searches/PM/Database/PM.csv")
EM_2 = pd.read_csv("Keyword Searches/EM2/Database/EM2.csv")
GW_ABM.shape
SH.shape
TC.shape
PM.shape
EM_2.shape

(40, 31)

(267, 34)

(128, 34)

(349, 36)

(786, 36)

In [5]:
list = ["AU","TI","SO","JI","AB","DE","ID","LA","DT","DT2","TC","CR","C1","DI","AR","FU","SN","PN","PP","PU","VL","PY",
        "RP","DB","AU_UN","AU1_UN","AU_UN_NR","SR_FULL","SR","CR_AU","CR_SO","AU_CO","AU1_CO", "BN"]

# First we create an empty dataframe and paste all databases on it
db_combined = pd.DataFrame(columns = list) 
db_combined = db_combined.append(GW_ABM, ignore_index=True)
db_combined = db_combined.append(SH, ignore_index=True)
db_combined = db_combined.append(TC, ignore_index=True)
db_combined = db_combined.append(PM, ignore_index = True)
#db_combined = db_combined.append(EM, ignore_index = True)
db_combined = db_combined.append(EM_2, ignore_index = True)
print("Database combined dimensions are: ", db_combined.shape)

db_combined = db_combined.drop_duplicates(subset = "TI")
print("After removing duplicates from titles, database dimensions are: ", db_combined.shape)
#print(db_combined.isna().sum())

db_combined_nulls = db_combined[db_combined['DI'].isnull()]
print("missing DI values: ", db_combined_nulls.shape)
db_combined_not_nulls = db_combined[db_combined['DI'].notnull()]
print("actual DI: ", db_combined_not_nulls.shape)

# Then we drop duplicates through the DI columns, by filtering only through non-null values
filt = db_combined_not_nulls.drop_duplicates(subset = "DI")
filt = filt.append(db_combined_nulls, ignore_index = True)
print("removing duplicates through DI leaves: ", filt.shape)

db_combined = filt.copy()
print("Cleaned combined database dimensions are: ", db_combined.shape)

Database combined dimensions are:  (1570, 37)
After removing duplicates from titles, database dimensions are:  (1558, 37)
missing DI values:  (46, 37)
actual DI:  (1512, 37)
removing duplicates through DI leaves:  (1556, 37)
Cleaned combined database dimensions are:  (1556, 37)


/Users/macadmin/anaconda3/envs/SALAR/lib/python3.8/site-packages/pandas/core/frame.py:7134: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [6]:
merged = db_combined.copy()
print('Merged Database dimensions are: ', merged.shape)
e_path = "Keyword Searches/Merged DB (GW-ABM,SH,PM,TC,EM)/Database/Merged DB (GW-ABM,SH,PM,TC,EM).csv"
merged.to_csv(e_path, index = False)
#check = pd.read_csv(e_path)
#check

Merged Database dimensions are:  (1556, 37)


## **Final treatment and export to bibliometrix**

In [ ]:
# Here we obtain the tags that must not be consider while reupdating the database to bibliometrix
df_tags = pd.Series(df.columns.values)
list = ["AB", "AR", "AU", "AU_UN", "AU_UN_NR", "AU1_UN", "BN", "C1", "CR", "DB", "DE", "DI", "DT", 
                     "DT2", "FU", "ID", "JI", "LA", "PN", "PP", "PU", "PY", "RP", "SN", "SO", "SR", "SR_FULL", 
                     "TC", "TI", "VL", "CR_AU", "CR_SO", "AU_CO", "AU1_CO"]
official_wos_tags = pd.Series(list)
tags_concat = pd.concat([df_tags, official_wos_tags])
tags_concat = tags_concat.sort_index()
tags_concat = tags_concat.drop_duplicates(keep = False)
print("To reupdate database to bibliometrix, the next columns must not be consider: ")
tags_concat

In [ ]:
# Final export to bibliometrix
db_combined.to_csv('Database.csv', index = False)
asd = pd.read_csv("Database.csv")
#print(asd.isna().sum())
asd.shape
#asd['DE']

## **IV - Export dataframe DOI identifier to Zotero**

In [ ]:
# Here we transform the column "DI" (DOI Identifier) to a text file in order to import it to zotero through the "add item
# by identifier option". For this, manually copy the files in the text file to this option (magic wand) in zotero.
# NaN files in DOI must be added manually to Zotero
list_zotero = GW_ABM['DI'].tolist()
list_zotero = [x for x in list_zotero if str(x) != 'nan']
with open('Zotero/list_zotero.txt', 'w') as file:
    for listitem in list_zotero:
        file.write('%s\n' % listitem)

#df[['DI', 'TI']]
#df2 = df.loc[df['DI'] == "NaN"]
#df2


In [ ]:
df.head()

In [ ]:
GW_ABM[['TI',"DI"]]
#pd.isna(GW_ABM["DI"].iloc[i])
df2 = GW_ABM.loc[GW_ABM['DI'] == "NaN"]
df2

In [ ]:
import pandas as pd
import os
os.chdir("/Users/macadmin/Google Drive/GW-ABM — Review/Bibliometric Analysis/")
df = pd.read_csv('FL_ABM_DB.csv')
pd.set_option('display.max_columns', 100)  # To show all columns on results
pd.set_option('display.max_rows', 5000) 
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)
df["DE"]